<a href="https://colab.research.google.com/github/ufbfung/pgx/blob/main/pgx_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
This is a development notebook used to create new features based primarily off the CPIC API.

## Data Model

|Data Model|Table|API endpoint|
|--|--|--|
|Gene|gene|/gene|
|Drug|drug|/drug|
|Allele|allele|/allele|
|Allele|allele_definition|/allele_definition|
|Guideline|guideline|/guideline|
|Publication|publication|/publication|
|Recommendation|recommendation|/recommendation|
|Test Alert|test_alert|/test_alert|
|Gene Results and Diplotypes|gene_result|/gene_result|
|Gene Results and Displotypes|diplotype|/diplotype|
|Pair|pair_view|/pair_view|
|Pair|pair|/pair|
|Allele Frequency|population_frequency_view|/population_frequency_view|

# Development

## Install dependencies

In [54]:
# install dependencies
!pip install streamlit

# libraries
import requests # for apis
import json # for json files
import streamlit as st

## Code

In [98]:
# Define static parameters

# CPIC API URLs
cpic_api_url = "https://api.cpicpgx.org/v1/"
drug_api_url = cpic_api_url + "drug"
guideline_api_url = cpic_api_url + "guideline"
recommendation_view_api_url = cpic_api_url + "recommendation_view"

# Functions that collect information from the user

def get_ethnicity():
  # Prompt user for ethnicity
  choices = ['Latino', 'American', 'European', 'Oceanian', 'East Asian',
          'Near Eastern', 'Central/South Asian', 'Sub-Saharan African',
          'African American/Afro-Caribbean', 'Other', 'Mixed Ethnicity',
          'Unknown']
  print("Please select one of the choices listed below")
  for c in choices:
    print(c)
  ethnicity = input("Ethnicity: ")
  while ethnicity not in choices:
    print("Invalid choice. Please try again.")
    ethnicity = input("Ethnicity: ")
  return ethnicity

def get_drug():
  # Prompt user for drug
  choices = ['codeine', 'abacavir', 'simvastatin']
  print("Please select one of the choices listed below")
  for c in choices:
    print(c)
  drug = input("Drug: ")
  while drug not in choices:
      print("Invalid choice. Please try again.")
      drug = input("Drug: ")
  return drug

def get_lookup_keys_for_query(drug):
    # Get lookup keys for the specified drug
    lookup_keys = get_lookup_keys_for_drug(drug)

    if lookup_keys is not None:
        print(f"Available lookup keys for {drug}:")
        for key in lookup_keys:
            print(key)

        # Prompt user for lookup key
        lookup_key = input("Select a lookup key: ")

        # Validate user input for lookup key
        while lookup_key not in lookup_keys:
            print("Invalid choice. Please try again.")
            lookup_key = input("Select a lookup key: ")

        # Get lookup values for the selected key
        lookup_values = get_lookup_values_for_key(drug, lookup_key)

        if lookup_values is not None:
            print(f"Available lookup values for {lookup_key}: {', '.join(lookup_values)}")

            # Prompt user for lookup value
            lookup_value = input(f"Select a lookup value for {lookup_key}: ")

            # Validate user input for lookup value
            while lookup_value not in lookup_values:
                print("Invalid choice. Please try again.")
                lookup_value = input(f"Select a lookup value for {lookup_key}: ")

            return lookup_key, lookup_value

    print("Failed to retrieve lookup keys.")
    return None

def get_lookup_keys_for_drug(drug):
    # Construct API URL with specified drug name
    url = f"{recommendation_view_api_url}?drugname=eq.{drug}"

    try:
        # Make the API request
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse and extract lookup keys and values from the JSON data
            data = response.json()
            lookup_keys_values = {}

            for recommendation in data:
                lookup_key_values = recommendation.get("lookupkey", {})
                for key, value in lookup_key_values.items():
                    if key not in lookup_keys_values:
                        lookup_keys_values[key] = set()
                    lookup_keys_values[key].add(value)

            return lookup_keys_values

        else:
            # Print an error message if the request was not successful
            print(f"Error: {response.status_code} - {response.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Print an error message if an exception occurs during the request
        print(f"Error: {e}")
        return None

def get_lookup_values_for_key(drug, lookup_key):
    # Construct API URL with specified drug name and lookup key
    url = f"{recommendation_view_api_url}?drugname=eq.{drug}"

    try:
        # Make the API request
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse and extract lookup values for the specified key from the JSON data
            data = response.json()
            lookup_values = set()

            for recommendation in data:
                lookup_key_values = recommendation.get("lookupkey", {})
                value = lookup_key_values.get(lookup_key)
                if value:
                    lookup_values.add(value)

            return lookup_values

        else:
            # Print an error message if the request was not successful
            print(f"Error: {response.status_code} - {response.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Print an error message if an exception occurs during the request
        print(f"Error: {e}")
        return None

def get_guidelineid_from_drug(json_data):
  try:
      # Load the JSON data
      data_list = json_data

      # Extract the guideline ID from the first element in the list
      if data_list and isinstance(data_list, list) and len(data_list) > 0:
          guideline_id = data_list[0].get("guidelineid")
          return guideline_id
      else:
          # Print an error message if the list is empty
          print("Error: Empty or invalid JSON list.")
          return None
  except json.JSONDecodeError as e:
      # Handle JSON decoding error
      print(f"Error decoding JSON: {e}")
      return None

  except Exception as e:
      # Handle other exceptions
      print(f"Error: {e}")
      return None

def get_drug_data(drug):
  # construct API URL with specified drug name
  url = f"{drug_api_url}?name=eq.{drug}"

  try:
      # Make the API request
      response = requests.get(url)

      # Check if the request was successful (status code 200)
      if response.status_code == 200:
        # Parse and return the JSON data
          return response.json()

      else:
        # Print an error message if request was not successful
        print(f"Error: {response.status_code} - {response.text}")
        return None

  except requests.exceptions.RequestException as e:
      # Print an error message if an exception occurs during the request
      print(f"Error: {e}")
      return None

def get_guideline(guideline_id):
    # construct API URL with specified drug name
    url = f"{guideline_api_url}?id=eq.{guideline_id}"

    try:
      # Make the API request
      response = requests.get(url)

      # Check if the request was successful (status code 200)
      if response.status_code == 200:
        # Parse and return the JSON data
          return response.json()

      else:
        # Print an error message if request was not successful
        print(f"Error: {response.status_code} - {response.text}")
        return None

    except requests.exceptions.RequestException as e:
        # Print an error message if an exception occurs during the request
        print(f"Error: {e}")
        return None
def get_guideline_for_specific_drug(drug):
    # Query drug api for a particular drug
    drug_data = get_drug_data(drug)

    # Get associated guidelines
    guideline_id = get_guidelineid_from_drug(drug_data)

    # Get information about the specific guideline to review
    guideline_data = get_guideline(guideline_id)

    # Print information about guidelines
    if guideline_data:
        print("Retrieved Guideline Data:")
        print(guideline_data)
    else:
        print("Error retrieving guideline data.")

def get_recommendation_for_specific_drug(drug, gene, phenotype):
    # construct API URL with specified drug name
      url = f"{recommendation_view_api_url}?drugname=eq.{drug}&lookupkey=cs.{{%22{gene}%22:%20%22{phenotype}%22}}"

      try:
        # Make the API request
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
          # Parse and return the JSON data
            print("CPIC Recommendations:")
            print(response.json())

        else:
          # Print an error message if request was not successful
          print(f"Error: {response.status_code} - {response.text}")
          return None

      except requests.exceptions.RequestException as e:
          # Print an error message if an exception occurs during the request
          print(f"Error: {e}")
          return None

if __name__ == "__main__":
    # Call the ethnicity function
    # ethnicity = get_ethnicity()
    drug = get_drug()
    gene, phenotype = get_lookup_keys_for_query(drug)

    # Get recommendations for specific drug
    get_recommendation_for_specific_drug(drug, gene, phenotype)

Please select one of the choices listed below
codeine
abacavir
simvastatin
Drug: simvastatin
Available lookup keys for simvastatin:
SLCO1B1
Select a lookup key: SLCO1B1
Available lookup values for SLCO1B1: Decreased Function, Increased Function, Normal Function, Possible Decreased Function, Poor Function, Indeterminate
Select a lookup value for SLCO1B1: Decreased Function
CPIC Recommendations:
[{'recommendationid': 3680391, 'lookupkey': {'SLCO1B1': 'Decreased Function'}, 'drugname': 'simvastatin', 'guidelinename': 'SLCO1B1, ABCG2, CYP2C9, and Statins', 'guidelineurl': 'https://cpicpgx.org/guidelines/cpic-guideline-for-statins/', 'implications': {'SLCO1B1': 'Increased simvastatin acid exposure as compared to normal function; increased risk of myopathy'}, 'drugrecommendation': 'Prescribe an alternative statin depending on the desired potency (see Figure 1 of PMID: 35152405 for recommendations for alternative statins). If simvastatin therapy is warranted, limit dose to <20mg/day.', 'class